# 天池o2o优惠券使用预测

## 加载相关库

In [1]:
# Import libraries necessary for this project
import pandas as pd
import numpy as np

## 导入数据

In [2]:
df_offline = pd.read_csv('./data/ccf_offline_stage1_train.csv', keep_default_na=False)
df_online = pd.read_csv('./data/ccf_online_stage1_train.csv', keep_default_na=False)
df_test = pd.read_csv('./data/ccf_offline_stage1_test_revised.csv', keep_default_na=False)

In [3]:
df_offline.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,null,null,0,null,20160217
1,1439408,4663,11002,150:20,1,20160528,null
2,1439408,2632,8591,20:1,0,20160217,null
3,1439408,2632,1078,20:1,0,20160319,null
4,1439408,2632,8591,20:1,0,20160613,null


In [4]:
df_offline.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754884 entries, 0 to 1754883
Data columns (total 7 columns):
User_id          int64
Merchant_id      int64
Coupon_id        object
Discount_rate    object
Distance         object
Date_received    object
Date             object
dtypes: int64(2), object(5)
memory usage: 93.7+ MB


## 简单统计

简单统计一下用户使用优惠券的情况

In [5]:
receive_and_buy = df_offline[(df_offline['Date_received'] != 'null') & (df_offline['Date'] != 'null')]
receive_not_buy = df_offline[(df_offline['Date_received'] != 'null') & (df_offline['Date'] == 'null')]
not_receive_and_buy = df_offline[(df_offline['Date_received'] == 'null') & (df_offline['Date'] != 'null')]
not_receive_not_buy = df_offline[(df_offline['Date_received'] == 'null') & (df_offline['Date'] == 'null')]

In [6]:
print('有优惠券，购买商品：{}'.format(receive_and_buy.shape[0]))
print('有优惠券，未购商品：{}'.format(receive_not_buy.shape[0]))
print('无优惠券，购买商品：{}'.format(not_receive_and_buy.shape[0]))
print('无优惠券，未购商品：{}'.format(not_receive_not_buy.shape[0]))

有优惠券，购买商品：75382
有优惠券，未购商品：977900
无优惠券，购买商品：701602
无优惠券，未购商品：0


可见，很多人（701602）购买商品却没有使用优惠券，也有很多人（977900）有优惠券但却没有使用，真正使用优惠券购买商品的人（75382）很少！所以，这个比赛的意义就是把优惠券送给真正可能会购买商品的人。

## 特征提取

In [7]:
print(df_offline.columns)

Index(['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance',
       'Date_received', 'Date'],
      dtype='object')


### 打折率 Discount_rate

In [8]:
df_offline['Discount_rate'].unique()

array(['null', '150:20', '20:1', '200:20', '30:5', '50:10', '10:5',
       '100:10', '200:30', '20:5', '30:10', '50:5', '150:10', '100:30',
       '200:50', '100:50', '300:30', '50:20', '0.9', '10:1', '30:1',
       '0.95', '100:5', '5:1', '100:20', '0.8', '50:1', '200:10',
       '300:20', '100:1', '150:30', '300:50', '20:10', '0.85', '0.6',
       '150:50', '0.75', '0.5', '200:5', '0.7', '30:20', '300:10', '0.2',
       '50:30', '200:100', '150:5'], dtype=object)

打折率分为 3 种情况：

- 'null' 表示没有打折: -1
- [0,1] 表示折扣率: 0
- x:y 表示满x减y: 1

处理方式：

- 打折类型：getDiscountType()
- 折扣率：convertRate()
- 满多少：getDiscountTarget()
- 减多少：getDiscountReduce()

**Build functions to deal with the Discount_rate**

In [9]:
def getDiscountType(row):
    if row == 'null':
        return -1
    elif ':' in row:
        return 1
    else:
        return 0

In [10]:
def convertRate(row):
    if row == 'null':
        return 1.0
    elif ':' in row:
        rows = row.split(':')
        return 1 - float(rows[1])/float(rows[0])
    else:
        return float(row)

In [11]:
def getDiscountTarget(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[0])
    else:
        return 0

In [12]:
def getDiscountReduce(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[1])
    else:
        return 0

In [13]:
def processData(df):
    df['Discount_type'] = df['Discount_rate'].apply(getDiscountType)
    df['Convert_rate'] = df['Discount_rate'].apply(convertRate)
    df['Discount_target'] = df['Discount_rate'].apply(getDiscountTarget)
    df['Discount_reduce'] = df['Discount_rate'].apply(getDiscountReduce)
    
    return df

In [14]:
df_offline = processData(df_offline)
df_test = processData(df_test)

In [15]:
df_offline = df_offline.drop(['Discount_rate'], axis=1)
df_test = df_test.drop(['Discount_rate'], axis=1)

In [16]:
df_offline.sample(10)

,User_id,Merchant_id,Coupon_id,Distance,Date_received,Date,Discount_type,Convert_rate,Discount_target,Discount_reduce
727054,6322328,5341,111,null,20160207,null,1,0.833333,30,5
65374,1793510,5717,8192,0,20160522,null,1,0.750000,20,5
435538,2406057,308,null,0,null,20160225,-1,1.000000,0,0
1064733,6760275,2934,null,0,null,20160201,-1,1.000000,0,0
918889,3457867,4083,4270,0,20160522,20160601,1,0.750000,20,5
1129369,536648,2099,null,9,null,20160605,-1,1.000000,0,0
478765,7015207,7974,null,0,null,20160128,-1,1.000000,0,0
979758,3832867,1469,null,1,null,20160614,-1,1.000000,0,0
1649229,4434731,8693,null,0,null,20160618,-1,1.000000,0,0
272200,4910933,6454,14031,1,20160130,null,1,0.900000,100,10


### 距离 Distance

In [17]:
df_offline['Distance'].unique()

array(['0', '1', 'null', '2', '10', '4', '7', '9', '3', '5', '6', '8'],
      dtype=object)

- 将距离数据类型由str转换为int
- 将null转换为-1

In [18]:
df_offline['Distance'] = df_offline['Distance'].replace('null', -1).astype(int) 
df_test['Distance'] = df_test['Distance'].replace('null', -1).astype(int)

In [19]:
df_offline.sample(10)

,User_id,Merchant_id,Coupon_id,Distance,Date_received,Date,Discount_type,Convert_rate,Discount_target,Discount_reduce
906109,3729181,2934,null,1,null,20160318,-1,1.000000,0,0
1254475,1692408,3381,7610,0,20160415,null,1,0.900000,200,20
1675100,4388051,517,null,0,null,20160512,-1,1.000000,0,0
1743108,5046936,2376,null,0,null,20160106,-1,1.000000,0,0
785609,1262422,7483,null,4,null,20160321,-1,1.000000,0,0
451810,6615494,7132,null,0,null,20160326,-1,1.000000,0,0
91552,3649016,1232,2533,1,20160603,null,1,0.800000,100,20
65379,864243,2080,null,1,null,20160508,-1,1.000000,0,0
259493,7223945,8775,null,0,null,20160321,-1,1.000000,0,0
1566137,5659698,7573,10517,-1,20160501,null,1,0.966667,30,1


### 领券日期 Date_received

In [20]:
date_received = df_offline['Date_received'].unique()
date_received = sorted(date_received[date_received != 'null'])

In [21]:
date_buy = df_offline['Date'].unique()
date_buy = sorted(date_buy[date_buy != 'null'])

In [22]:
print('优惠卷收到日期从 {} 到 {}'.format(date_received[0], date_received[-1]))
print('消费日期从 {} 到 {}'.format(date_buy[0], date_buy[-1]))

优惠卷收到日期从 20160101 到 20160615
消费日期从 20160101 到 20160630


关于领劵日期的特征：
- weekday : {null, 1, 2, 3, 4, 5, 6, 7}
- weekday_type : {1, 0}（周六和周日为1，其他为0）
- weekday_1 : {1, 0, 0, 0, 0, 0, 0}
- weekday_2 : {0, 1, 0, 0, 0, 0, 0}
- weekday_3 : {0, 0, 1, 0, 0, 0, 0}
- weekday_4 : {0, 0, 0, 1, 0, 0, 0}
- weekday_5 : {0, 0, 0, 0, 1, 0, 0}
- weekday_6 : {0, 0, 0, 0, 0, 1, 0}
- weekday_7 : {0, 0, 0, 0, 0, 0, 1}

In [23]:
from datetime import date

In [24]:
def getWeekday(row):
    if row == 'null':
        return row
    else:
        return date(int(row[:4]), int(row[4:6]), int(row[6:])).weekday() + 1

In [25]:
df_offline['weekday'] = df_offline['Date_received'].astype(str).apply(getWeekday)
df_test['weekday'] = df_test['Date_received'].astype(str).apply(getWeekday)

In [26]:
df_offline['weekday_type'] = df_offline['weekday'].apply(lambda x: 1 if x in [6,7] else 0)
df_test['weekday_type'] = df_test['weekday'].apply(lambda x: 1 if x in [6,7] else 0)

In [27]:
# Change weekday to one_hot encoding
col_names = ['weekday_' + str(i) for i in range(1,8)]

tmp_df = pd.get_dummies(df_offline['weekday'].replace('null', np.nan))
tmp_df.columns = col_names
df_offline[col_names] = tmp_df

tmp_df = pd.get_dummies(df_test['weekday'].replace('null', np.nan))
tmp_df.columns = col_names
df_test[col_names] = tmp_df

In [28]:
df_offline = df_offline.drop(['weekday'], axis=1)
df_test = df_test.drop(['weekday'], axis=1)

In [29]:
df_offline.sample(10)

,User_id,Merchant_id,Coupon_id,Distance,Date_received,Date,Discount_type,Convert_rate,Discount_target,Discount_reduce,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7
1536991,2598323,8775,null,0,null,20160306,-1,1.000000,0,0,0,0,0,0,0,0,0,0
199367,1235285,3314,null,0,null,20160512,-1,1.000000,0,0,0,0,0,0,0,0,0,0
859969,6581137,1274,null,8,null,20160507,-1,1.000000,0,0,0,0,0,0,0,0,0,0
872404,4731997,6901,null,1,null,20160128,-1,1.000000,0,0,0,0,0,0,0,0,0,0
243624,2920178,6803,10649,0,20160324,null,1,0.833333,30,5,0,0,0,0,1,0,0,0
84012,5416920,3338,null,0,null,20160220,-1,1.000000,0,0,0,0,0,0,0,0,0,0
1737773,1812386,2099,12034,10,20160204,null,1,0.900000,100,10,0,0,0,0,1,0,0,0
927833,3512173,450,8555,3,20160202,null,1,0.833333,30,5,0,0,1,0,0,0,0,0
310330,5171657,4582,null,3,null,20160525,-1,1.000000,0,0,0,0,0,0,0,0,0,0
1488467,4175117,7555,9871,5,20160420,null,1,0.833333,30,5,0,0,0,1,0,0,0,0


所有特征：
- Convert_rate
- Discount_type
- Discount_target
- Discount_reduce
- Distance
- weekday_type
- weekday_1
- weekday_2
- weekday_3
- weekday_4
- weekday_5
- weekday_6
- weekday_7

## 标签标注

三种情况：

- Date_received == 'null'：表示没有领到优惠券，无需考虑，y = -1

- (Date_received != 'null') & (Date != 'null') & (Date - Date_received <= 15)：表示领取优惠券且在15天内使用，即正样本，y = 1

- (Date_received != 'null') & ((Date == 'null') | (Date - Date_received > 15))：表示领取优惠券未在在15天内使用，即负样本，y = 0

定义标签备注函数：

In [30]:
def label(row):
    if row['Date_received'] == 'null':
        return int(-1)
    if row['Date'] != 'null':
        td = pd.to_datetime(row['Date'], format='%Y%m%d') - pd.to_datetime(row['Date_received'], format='%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return int(1)
    return int(0)

In [31]:
df_offline['label'] = df_offline.apply(label, axis=1)

In [32]:
print(df_offline['label'].value_counts())

 0    988887
-1    701602
 1     64395
Name: label, dtype: int64


In [33]:
df_offline.sample(10)

,User_id,Merchant_id,Coupon_id,Distance,Date_received,Date,Discount_type,Convert_rate,Discount_target,Discount_reduce,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,label
1684798,6023531,4149,null,0,null,20160601,-1,1.000000,0,0,0,0,0,0,0,0,0,0,-1
6869,4959960,4860,8105,10,20160201,null,1,0.700000,100,30,0,1,0,0,0,0,0,0,0
307396,6200729,8035,null,1,null,20160622,-1,1.000000,0,0,0,0,0,0,0,0,0,0,-1
106674,3158030,2934,null,0,null,20160520,-1,1.000000,0,0,0,0,0,0,0,0,0,0,-1
1507092,5671578,8605,null,0,null,20160513,-1,1.000000,0,0,0,0,0,0,0,0,0,0,-1
1084717,4390035,2146,11173,1,20160130,null,1,0.900000,100,10,1,0,0,0,0,0,1,0,0
838694,4341313,3381,12821,0,20160125,null,1,0.900000,200,20,0,1,0,0,0,0,0,0,0
1569068,3638819,1469,null,0,null,20160519,-1,1.000000,0,0,0,0,0,0,0,0,0,0,-1
1142881,6615664,4663,818,7,20160124,null,1,0.866667,150,20,1,0,0,0,0,0,0,1,0
406285,6050954,3381,null,10,null,20160119,-1,1.000000,0,0,0,0,0,0,0,0,0,0,-1


## 建立线性模型 SGDClassifier

- 使用上面提取的14个特征。

- 训练集：20160101-20160515；验证集：20160516-20160615。

- 用线性模型 SGDClassifier

### 划分训练集/验证集

In [38]:
df = df_offline[df_offline['label'] != -1].copy()
train = df[(df['Date_received'] < '20160516')].copy()
test = df[(df['Date_received'] >= '20160516') & (df['Date_received'] <= '20160615')].copy()

In [39]:
train['label'].value_counts()

0    759172
1     41524
Name: label, dtype: int64

In [40]:
test['label'].value_counts()

0    229715
1     22871
Name: label, dtype: int64

### 特征数量

In [41]:
factors = ['Convert_rate', 'Discount_type', 'Discount_target', 'Discount_reduce', 'Distance', 'weekday_type',
          'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']
print('共有{}个特征'.format(len(factors)))

共有13个特征


### 建立模型

In [42]:
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve
from sklearn.preprocessing import MinMaxScaler

In [43]:
def check_model(data, factors):
    
    classifier = lambda: SGDClassifier(
        loss='log',  # loss function: logistic regression
        penalty='elasticnet', # L1 & L2
        fit_intercept=True,  # 是否存在截距，默认存在
        max_iter=100, 
        shuffle=True,  # Whether or not the training data should be shuffled after each epoch
        n_jobs=-1, # The number of processors to use
        class_weight=None) # Weights associated with classes. If not given, all classes are supposed to have weight one.
 
    # 管道机制使得参数集在新数据集（比如测试集）上的重复使用，管道机制实现了对全部步骤的流式化封装和管理。
    model = Pipeline(steps=[
        ('ss', StandardScaler()), # transformer
        ('en', classifier())  # estimator
    ])
 
    parameters = {
        'en__alpha': [ 0.001, 0.01, 0.1],
        'en__l1_ratio': [ 0.001, 0.01, 0.1]
    }
 
    # StratifiedKFold用法类似Kfold，但是他是分层采样，确保训练集，测试集中各类别样本的比例与原始数据集中相同。
    folder = StratifiedKFold(n_splits=5, shuffle=True)
    
    # Exhaustive search over specified parameter values for an estimator.
    grid_search = GridSearchCV(
        model, 
        parameters, 
        cv=folder, 
        n_jobs=-1,  # -1 means using all processors
        verbose=1)
    
    grid_search = grid_search.fit(data[factors], 
                                  data['label'])
    
    return grid_search

### 训练

In [44]:
model = check_model(train, factors)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  6.6min finished
/Users/ryne/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/ryne/anaconda3/lib/python3.7/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/Users/ryne/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol

### 验证

对验证集中每个优惠券预测的结果计算 AUC，再对所有优惠券的 AUC 求平均。计算 AUC 的时候，如果 label 只有一类，就直接跳过，因为 AUC 无法计算。

In [45]:
y_pred = model.predict_proba(test[factors])

/Users/ryne/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:381: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [46]:
test_tmp = test.copy()
test_tmp['pred_prob'] = y_pred[:, 1]

In [47]:
test_tmp.sample(10)

,User_id,Merchant_id,Coupon_id,Distance,Date_received,Date,Discount_type,Convert_rate,Discount_target,Discount_reduce,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,label,pred_prob
1039603,2912497,760,13758,3,20160520,null,1,0.750000,20,5,0,0,0,0,0,1,0,0,0,0.067225
1175902,1529271,8021,12620,0,20160530,null,1,0.900000,10,1,0,1,0,0,0,0,0,0,0,0.117151
739586,190694,450,1613,1,20160601,null,1,0.833333,30,5,0,0,0,1,0,0,0,0,0,0.103562
1412774,1500537,1080,11378,0,20160517,null,1,0.950000,20,1,0,0,1,0,0,0,0,0,0,0.133804
26497,2397068,4759,13552,0,20160526,null,1,0.900000,50,5,0,0,0,0,1,0,0,0,0,0.099307
624402,5141324,6901,2366,0,20160521,null,1,0.833333,30,5,1,0,0,0,0,0,1,0,0,0.115482
1457417,4313157,450,1613,0,20160603,null,1,0.833333,30,5,0,0,0,0,0,1,0,0,0,0.102182
1663385,6684695,760,2418,0,20160605,null,1,0.833333,30,5,1,0,0,0,0,0,0,1,0,0.090458
140271,4347024,3621,2705,0,20160522,null,1,0.750000,20,5,1,0,0,0,0,0,0,1,0,0.110402
1134816,3184426,760,2418,2,20160612,null,1,0.833333,30,5,1,0,0,0,0,0,0,1,0,0.059509


#### 计算AUC

In [54]:
test_group = test_tmp.groupby(['Coupon_id'])
aucs = []
for i in test_group:
    tmp_df = i[1] 
    if len(tmp_df['label'].unique()) != 2:
        continue
    fpr, tpr, thresholds = roc_curve(tmp_df['label'], tmp_df['pred_prob'], pos_label=1)
    aucs.append(auc(fpr, tpr))
print(np.average(aucs))

0.5323444694516164


## 测试

In [57]:
# test prediction for submission
y_test_pred = model.predict_proba(df_test[factors])
df_result = df_test[['User_id','Coupon_id','Date_received']].copy()
df_result['Probability'] = y_test_pred[:,1]
df_result.to_csv('submit1.csv', index=False, header=False)

/Users/ryne/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:381: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [58]:
df_result.sample(10)

,User_id,Coupon_id,Date_received,Probability
71739,845503,13602,20160721,0.014963
6966,2676859,9983,20160722,0.035442
19376,943076,7083,20160727,0.068467
88736,5525831,3629,20160724,0.025390
1241,6224530,9983,20160729,0.102182
15531,1238679,10418,20160703,0.090458
53276,1738645,13602,20160719,0.128350
61608,1781245,3780,20160705,0.017199
69836,1508197,9983,20160711,0.011036
6688,7066846,13602,20160712,0.105106


## 保存模型 & 导入模型

In [61]:
import os
import pickle

In [62]:
if not os.path.isfile('1_model.pkl'):
    with open('1_model.pkl', 'wb') as f:
        pickle.dump(model, f)
else:
    with open('1_model.pkl', 'rb') as f:
        model = pickle.load(f)

/Users/ryne/anaconda3/lib/python3.7/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator StandardScaler from version 0.19.0 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/ryne/anaconda3/lib/python3.7/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator SGDClassifier from version 0.19.0 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/ryne/anaconda3/lib/python3.7/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator Pipeline from version 0.19.0 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/ryne/anaconda3/lib/python3.7/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator GridSearchCV from version 0.19.0 when using version 0.20.2. This might lead to breaking code or inv